# Verification of New Results for Bots Case (revised, alternate example)
#### Unified Model with Integrated Bot 

James Yu, 14 January 2023

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
np.set_printoptions(suppress=True) # disable scientific notation for readability

In [2]:
def infinite_solution(A_n, A_c, delta, c, x_0, z, b = np.identity(3)):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    O = np.zeros((n, n))
    
    Q = np.block([
        [I, O], 
        [O, I]
    ])
    A = np.block([
        [A_n, A_c], 
        [O, I]
    ])
    B = np.block([
        [b], # identity matrix by default 
        [O]
    ])
    K_t = Q
    K_sequence = [K_t]
    
    while True: # generate solution matrices; note I divide delta into the inverse term for simplification
        K_t_new = Q + delta * A.T @ (K_t - K_t @ B @ inv(B.T @ K_t @ B + c * I / delta) @ B.T @ K_t) @ A
        K_sequence.insert(0, K_t_new)
        if np.allclose(K_t, K_t_new, rtol = eps, atol = eps): break
        K_t = K_t_new

    chi_var = np.block([
        [x_0], 
        [z]
    ])
    chi_ts = [chi_var]
    r_ts = []
    K_ss = K_sequence[0]
    L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []
    
    i = 0
    while True:
        r_t = L_ss @ chi_var
        r_ts.append(r_t)
        p = -(chi_var.T @ Q @ chi_var + c * r_t.T @ r_t).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = A @ chi_var + B @ r_t
        chi_ts.append(chi_var_new)
        if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        i += 1
        
    return chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss

# 0. Setup

In [3]:
A_1_n = 0.9 * np.array([
    [0.6, 0.3, 0.1],
    [0.4, 0.1, 0.5],
    [0.1, 0.7, 0.2]
])
A_1_c = np.diag([0.1, 0.1, 0.1])

delta = 0.54
c = 37.28
x_0 = np.array([[60.77, -4.3, -80.5]], ndmin = 2).T
z = np.array([[-6.0, 4.0, 3.0]], ndmin = 2).T

In [4]:
chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss = infinite_solution(A_1_n, A_1_c, delta, c, x_0, z)

In [5]:
print(K_ss)

[[1.37823398 0.2474061  0.21977945 0.07919491 0.06258287 0.0409958 ]
 [0.2474061  1.36724688 0.17508143 0.05301135 0.04464539 0.06544638]
 [0.21977945 0.17508143 1.2017931  0.03388671 0.053984   0.03762203]
 [0.07919491 0.05301135 0.03388671 2.20775135 0.01586067 0.01089185]
 [0.06258287 0.04464539 0.053984   0.01586067 2.19968782 0.01554776]
 [0.0409958  0.06544638 0.03762203 0.01089185 0.01554776 2.19625973]]


In [6]:
print(L_ss)

[[-0.01207064 -0.0075156  -0.00385728 -0.0030757  -0.00122795 -0.00088322]
 [-0.00906006 -0.00420749 -0.00947825 -0.00109161 -0.0025693  -0.00116958]
 [-0.00406659 -0.01181453 -0.00444743 -0.00078301 -0.0010076  -0.00224096]]


In [7]:
chi_ts[-1] # terminal opinions

array([[-1.03993199],
       [ 0.23483942],
       [ 0.42422861],
       [-6.        ],
       [ 4.        ],
       [ 3.        ]])

In [8]:
r_ts[-1] # terminal messages

array([[ 0.02004406],
       [-0.00282349],
       [-0.00648749]])

# 1. Expression for $K_1$ component of the block matrix $K^*$

Recall that this expression was:

$$
K_1 = I + c\delta(A^n)'K_1(\delta K_1 + cI)^{-1}A^n
$$

In [9]:
print(K_ss)

[[1.37823398 0.2474061  0.21977945 0.07919491 0.06258287 0.0409958 ]
 [0.2474061  1.36724688 0.17508143 0.05301135 0.04464539 0.06544638]
 [0.21977945 0.17508143 1.2017931  0.03388671 0.053984   0.03762203]
 [0.07919491 0.05301135 0.03388671 2.20775135 0.01586067 0.01089185]
 [0.06258287 0.04464539 0.053984   0.01586067 2.19968782 0.01554776]
 [0.0409958  0.06544638 0.03762203 0.01089185 0.01554776 2.19625973]]


In [10]:
print(K_ss[0:3,0:3]) # K_1

[[1.37823398 0.2474061  0.21977945]
 [0.2474061  1.36724688 0.17508143]
 [0.21977945 0.17508143 1.2017931 ]]


In [11]:
K_1 = K_ss[0:3,0:3]
LHS = np.identity(3) + c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n
np.allclose(LHS, K_1) # is approximately equal?

True

In [12]:
LHS

array([[1.37823398, 0.2474061 , 0.21977945],
       [0.2474061 , 1.36724688, 0.17508143],
       [0.21977945, 0.17508143, 1.2017931 ]])

They appear to be the same. To decompose:

In [13]:
c*delta*A_1_n.T @ K_1

array([[17.17377966, 12.9154834 ,  5.83546313],
       [10.72692557,  6.04244349, 16.75373693],
       [ 5.53475345, 13.46862441,  6.33910459]])

In [14]:
inv(delta*K_1 + c*np.identity(3)) 

array([[ 0.02629959, -0.00009221, -0.00008206],
       [-0.00009221,  0.0263036 , -0.00006528],
       [-0.00008206, -0.00006528,  0.02636549]])

In [15]:
c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n

array([[0.37823398, 0.2474061 , 0.21977945],
       [0.2474061 , 0.36724688, 0.17508143],
       [0.21977945, 0.17508143, 0.2017931 ]])

# 2. Expression for $K_2$ component of the block matrix $K^*$

$$
K_2 = c \delta {A^n}' (\delta K_1 + cI)^{-1} (K_1 A^z + K_2)
$$

In [16]:
print(K_ss)

[[1.37823398 0.2474061  0.21977945 0.07919491 0.06258287 0.0409958 ]
 [0.2474061  1.36724688 0.17508143 0.05301135 0.04464539 0.06544638]
 [0.21977945 0.17508143 1.2017931  0.03388671 0.053984   0.03762203]
 [0.07919491 0.05301135 0.03388671 2.20775135 0.01586067 0.01089185]
 [0.06258287 0.04464539 0.053984   0.01586067 2.19968782 0.01554776]
 [0.0409958  0.06544638 0.03762203 0.01089185 0.01554776 2.19625973]]


In [17]:
print(K_ss[0:3, 3:6]) # K_2

[[0.07919491 0.06258287 0.0409958 ]
 [0.05301135 0.04464539 0.06544638]
 [0.03388671 0.053984   0.03762203]]


In [18]:
K_2 = K_ss[0:3, 3:6]
LHS_2 = c * delta * A_1_n.T @ inv(c * np.identity(3) + delta * K_1) @ (K_2 + K_1 @ A_1_c)
np.allclose(LHS_2, K_2) # is approximately equal?

True

In [19]:
LHS_2

array([[0.07919491, 0.06258287, 0.0409958 ],
       [0.05301135, 0.04464539, 0.06544638],
       [0.03388671, 0.053984  , 0.03762203]])

# 3. Expression for $x_{\infty}$ terminal opinions

$$
x_\infty = (\delta K_1 + cI - cA^n)^{-1}(cA^z - \delta K_2)z
$$

In [20]:
LHS_3 = inv(delta * K_1 + c*np.identity(3) - c * A_1_n) @ (c * A_1_c - delta * K_2) @ z
LHS_3

array([[-1.03993199],
       [ 0.23483942],
       [ 0.42422861]])

In [21]:
chi_ts[-1][0:3]

array([[-1.03993199],
       [ 0.23483942],
       [ 0.42422861]])

In [22]:
np.allclose(chi_ts[-1][0:3], LHS_3)

True

Checking limit vanishing:

In [23]:
(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n)

array([[0.52792936, 0.2624844 , 0.08614272],
       [0.35093994, 0.08579251, 0.44052175],
       [0.08593341, 0.61818547, 0.17555257]])

In [24]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 2)

array([[0.3782282 , 0.21434459, 0.17622993],
       [0.25323505, 0.37180076, 0.14535911],
       [0.27739857, 0.18411591, 0.31054539]])

In [25]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 3)

array([[0.29004389, 0.22661094, 0.15794268],
       [0.27666116, 0.18822686, 0.21111884],
       [0.2377467 , 0.28058321, 0.15951997]])

In [26]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 100)

array([[0.00000086, 0.00000072, 0.00000056],
       [0.00000086, 0.00000072, 0.00000056],
       [0.00000086, 0.00000072, 0.00000056]])

In [27]:
np.linalg.matrix_power(c * inv(delta * K_1 + c * np.identity(3)) @ A_1_n, 1000)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])